<a href="https://colab.research.google.com/github/nedokormysh/GB_NLP_intro/blob/lesson13/NLP_intro_hw_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Модель BERT и GPT
Задание
Взять датасет
https://huggingface.co/datasets/merionum/ru_paraphraser

решить задачу парафраза

(дополнительно необязательная задача) на выбор взять

https://huggingface.co/datasets/sberquad

https://huggingface.co/datasets/blinoff/medical_qa_ru_data

натренировать любую модель для вопросно ответной системы

In [24]:
!pip install transformers sentencepiece -q

In [25]:
!pip install datasets -q

In [26]:
# Загрузка датасета

from datasets import load_dataset

corpus = load_dataset('merionum/ru_paraphraser')
corpus

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [27]:
corpus['train'][0:4]

{'id': ['1', '2', '3', '4'],
 'id_1': ['201', '202', '273', '220'],
 'id_2': ['8159', '8158', '8167', '8160'],
 'text_1': ['Полицейским разрешат стрелять на поражение по гражданам с травматикой.',
  'Право полицейских на проникновение в жилище решили ограничить.',
  'Президент Египта ввел чрезвычайное положение в мятежных городах.',
  'Вернувшихся из Сирии россиян волнует вопрос трудоустройства на родине.'],
 'text_2': ['Полиции могут разрешить стрелять по хулиганам с травматикой.',
  'Правила внесудебного проникновения полицейских в жилище уточнят.',
  'Власти Египта угрожают ввести в стране чрезвычайное положение.',
  'Самолеты МЧС вывезут россиян из разрушенной Сирии.'],
 'class': ['0', '0', '0', '-1']}

In [28]:
# Загрузка предобученных моделей и пайплайна

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rut5-base-paraphraser")

model = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-base-paraphraser")

In [29]:
# Функция парафраза

def paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [31]:
# Примеры работы парафраза

for i in range(10):
    print('Исходный текст: ', corpus["train"][i]['text_1'])
    print('Парафраз: ', paraphrase(corpus["train"][i]['text_1']))
    print('-' * 20)

Исходный текст:  Полицейским разрешат стрелять на поражение по гражданам с травматикой.
Парафраз:  Полицейские могут стрелять на нападение на граждан с травматизмом.
--------------------
Исходный текст:  Право полицейских на проникновение в жилище решили ограничить.
Парафраз:  Решили ограничить право полицейских проникнуть в жилище.
--------------------
Исходный текст:  Президент Египта ввел чрезвычайное положение в мятежных городах.
Парафраз:  Глава Египта объявил чрезвычайное положение в городах-мятежниках.
--------------------
Исходный текст:  Вернувшихся из Сирии россиян волнует вопрос трудоустройства на родине.
Парафраз:  Вопрос трудоустройства в Сирии волнует россиян, вернувшихся из Сирии.
--------------------
Исходный текст:  В Москву из Сирии вернулись 2 самолета МЧС с россиянами на борту.
Парафраз:  Из Сирии в Москву вернулись два самолета с гражданами России.
--------------------
Исходный текст:  Приставы соберут отпечатки пальцев российских должников.
Парафраз:  Приставы соб